In [57]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import scipy.optimize
import pandas.io.sql as psql
import pandas as pd
import sys
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests
import re



In [2]:
%run ../../../lib/dwh.py

In [3]:
con=get_DWH()

In [88]:
%run indebtedness.py

In [5]:
%cd  C:\Users\Sean Violante\Documents\Projects\lendico\projects\returns\src

C:\Users\Sean Violante\Documents\Projects\lendico\projects\returns\src


# XML

In [5]:
scoring=get_scoring(con)

In [7]:
loan_requests=get_loan_requests(con)

In [13]:
loan_requests1=extract_indebtedness(loan_requests)

In [14]:
loan_requests1['indebt:actual'].notnull().sum()

71

In [15]:
scoring.columns

Index([    u'dwh_country_id',                 u'id',            u'fk_user',
                  u'request',           u'response',      u'provider_name',
                u'is_failed', u'ignore_for_scoring',              u'score',
       u'data_accessible_by',         u'created_at',         u'updated_at',
              u'dwh_created',  u'dwh_last_modified',       u'dwh_row_hash',
                u'is_latest'],
      dtype='object')

In [18]:
scoring_ind=parse_scoring(scoring)

In [22]:
merkmale = extract_merkmale(scoring_ind)

In [19]:
score_merkmale=scoring[['id_loan_request',u'loan_request_nr',  u'fk_user','id', 
                        'is_failed', u'ignore_for_scoring',u'score', 'created_at' ]].\
merge(merkmale.reset_index(), on='id')

KeyError: "['id_loan_request' u'loan_request_nr'] not in index"

In [18]:
score_merkmale.to_clipboard(encoding='UTF-8')

In [23]:
merkmale.columns

Index([u'Beschreibung', u'Betrag', u'Betrag_waehrung', u'Datum',
       u'Kontonummer', u'Merkmal_eigenesMerkmal', u'Merkmal_id',
       u'Merkmal_merkmalOhneGeburtsdatum', u'Merkmal_typ', u'Merkmalcode',
       u'Ratenart', u'Ratenzahl', u'Date'],
      dtype='object')

#Not Creditworthy

In [28]:
merkmale1=calc_creditworthy(merkmale)

In [29]:
merkmale.columns

Index([u'Beschreibung', u'Betrag', u'Betrag_waehrung', u'Datum',
       u'Kontonummer', u'Merkmal_eigenesMerkmal', u'Merkmal_id',
       u'Merkmal_merkmalOhneGeburtsdatum', u'Merkmal_typ', u'Merkmalcode',
       u'Ratenart', u'Ratenzahl', u'Date', u'not_creditworthy',
       u'not_creditworthy_US', u'not_creditworthy_KRMLKWKX_US'],
      dtype='object')

KR, ML with US

In [23]:
RK=match_id(merkmale, merkmale.Merkmalcode.isin(['RK']))
RK.Merkmalcode.unique()

array(['XX', 'RK', 'ER'], dtype=object)

In [24]:
US_cases=match_id(merkmale, merkmale.Merkmalcode.isin(['US']))
US_cases.Merkmalcode.unique()

array(['KW', 'SD', 'US', 'SG', 'SE', 'ER', 'KR'], dtype=object)

In [25]:
US_cases[US_cases.Merkmal_typ=='hauptmerkmal'].Merkmalcode.unique()

array(['KW', 'KR'], dtype=object)

In [30]:
not_creditworthy_user=merkmale.groupby(level=['dwh_country_id','id'])['not_creditworthy',
                                                               'not_creditworthy_US',
                                                               'not_creditworthy_KRMLKWKX_US'].max()
# use max to find user with at least one row labelled not creditworthy
not_creditworthy_user.sum()

not_creditworthy                4393
not_creditworthy_US               18
not_creditworthy_KRMLKWKX_US     315
dtype: float64

In [31]:
not_creditworthy_user.shape

(7410, 3)

In [27]:
merkmale.Merkmal_typ[merkmale.Merkmalcode=='RK'].unique()

array(['untermerkmal'], dtype=object)

In [431]:
132/3431

0.03847274846983387

## Additional Limit

In [34]:
additional_limit_outstanding_user=calc_additional_limit(merkmale)

In [30]:
additional_limit_outstanding_user.head()

dwh_country_id  id   
1               97971    11000
                97973     7300
                97976     8000
                97991     2659
                97996     6000
Name: add_Betrag, dtype: float64

In [386]:
additional_limit_outstanding_user.head()

dwh_country_id  id   
1               96444     3000
                96450     1000
                96452     1200
                96462    40000
                96463     7700
Name: Betrag, dtype: float64

In [373]:
print additional_limit.shape
print additional_limit_paid_off.shape
print additional_limit_outstanding.shape

(2585, 13)
(635, 13)
(1950, 14)


## Line of Credit

In [363]:
match_id(merkmale, merkmale.Merkmalcode.isin(['KG'])).Merkmalcode.unique()

array(['GI', 'KG', 'XX'], dtype=object)

In [38]:
LOC_outstanding_user=calc_LOC(merkmale)

In [103]:
LOC_outstanding[LOC_outstanding.Merkmalcode=='KG']

Beschreibung  Betrag Betrag_waehrung Datum  \
dwh_country_id id    record                                              
1              99197 6         Girokonto     NaN             NaN   NaN   
               99210 1         Girokonto     NaN             NaN   NaN   
               99216 1         Girokonto     NaN             NaN   NaN   
                     2         Girokonto     NaN             NaN   NaN   
               99208 4         Girokonto     NaN             NaN   NaN   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 6              NaN                    NaN           5   
               99210 1              NaN                    NaN           2   
               99216 1              NaN                    NaN           2   
                     2              NaN                    NaN           3   
               99208 4              NaN                    NaN           4   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 6                                  NaN  hauptmerkmal   
               99210 1                                  NaN  hauptmerkmal   
               99216 1                                  NaN  hauptmerkmal   
                     2                                  NaN  hauptmerkmal   
               99208 4                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 6               GI      NaN       NaN            False   
               99210 1               GI      NaN       NaN            False   
               99216 1               GI      NaN       NaN            False   
                     2               GI      NaN       NaN            False   
               99208 4               GI      NaN       NaN            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99197 6                      NaN                          NaN   
               99210 1                      NaN                          NaN   
               99216 1                      NaN                          NaN   
                     2                      NaN                          NaN   
               99208 4                      NaN                          NaN   

                            paid  
dwh_country_id id    record       
1              99197 6       NaN  
               99210 1       NaN  
               99216 1       NaN  
                     2       NaN  
               99208 4       NaN

In [ ]:
# LOC can be time limited too?

In [101]:
LOC[LOC.Merkmalcode=='KG']

Beschreibung  Betrag Betrag_waehrung  \
dwh_country_id id     record                                                 
1              99219  11      Kredit zum Girokonto    9900             EUR   
               99278  1       Kredit zum Girokonto    6000             EUR   
               99315  7       Kredit zum Girokonto    5000             EUR   
               99313  6       Kredit zum Girokonto    1500             EUR   
               99333  2       Kredit zum Girokonto    1000             EUR   
               99354  1       Kredit zum Girokonto     475             EUR   
               99365  14      Kredit zum Girokonto    3400             EUR   
               99380  5       Kredit zum Girokonto     100             EUR   
               99392  12      Kredit zum Girokonto    3000             EUR   
                      14      Kredit zum Girokonto    3000             EUR   
               99412  8       Kredit zum Girokonto    1000             EUR   
               99414  4       Kredit zum Girokonto    2000             EUR   
               97137  19      Kredit zum Girokonto    3800             EUR   
               98002  2       Kredit zum Girokonto    1000             EUR   
               98139  2       Kredit zum Girokonto    1500             EUR   
               98239  25      Kredit zum Girokonto    2000             EUR   
               98242  2       Kredit zum Girokonto    3000             EUR   
               98705  3       Kredit zum Girokonto     500             EUR   
               98906  3       Kredit zum Girokonto    1000             EUR   
               98909  6       Kredit zum Girokonto    7700             EUR   
               99024  8       Kredit zum Girokonto    2000             EUR   
               99125  2       Kredit zum Girokonto    1500             EUR   
               96445  6       Kredit zum Girokonto    3800             EUR   
               96450  6       Kredit zum Girokonto     200             EUR   
               96481  5       Kredit zum Girokonto    1000             EUR   
               96482  7       Kredit zum Girokonto    1000             EUR   
               96495  2       Kredit zum Girokonto     200             EUR   
               96521  13      Kredit zum Girokonto     400             EUR   
               96522  5       Kredit zum Girokonto    2500             EUR   
               96526  1       Kredit zum Girokonto     700             EUR   
...                                            ...     ...             ...   
               100982 1       Kredit zum Girokonto    6400             EUR   
               100917 3       Kredit zum Girokonto    1000             EUR   
                      6       Kredit zum Girokonto     500             EUR   
               100957 3       Kredit zum Girokonto    1900             EUR   
               100967 6       Kredit zum Girokonto    1000             EUR   
               100954 4       Kredit zum Girokonto     500             EUR   
               100937 8       Kredit zum Girokonto   10000             EUR   
                      11      Kredit zum Girokonto    7000             EUR   
               101032 18      Kredit zum Girokonto     500             EUR   
               101043 2       Kredit zum Girokonto    7500             EUR   
               101001 13      Kredit zum Girokonto    5000             EUR   
               101090 3       Kredit zum Girokonto     500             EUR   
               101115 12      Kredit zum Girokonto     250             EUR   
               101117 12      Kredit zum Girokonto     250             EUR   
               101070 2       Kredit zum Girokonto    5500             EUR   
               101101 2       Kredit zum Girokonto     500             EUR   
               101062 9       Kredit zum Girokonto    2500             EUR   
               101008 6       Kredit zum Girokonto    2400             EUR   
               100925 12      Kredit zum Girokonto    1000  

In [105]:
LOC.shape

(4999, 15)

##Net Loans

In [32]:
net_loans=match_id(merkmale, merkmale.Merkmalcode.isin(['KR','ML']))
net_loans_exclude_US=drop_id(net_loans,net_loans.Merkmalcode=='US')
net_loans_exclude_US_outstanding=drop_paid_off(net_loans_exclude_US)


In [121]:
net_loans_exclude_US_outstanding.head()

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99197 4            Kredit    5364             EUR  01.09.2014   
                     5            Kredit   27197             EUR  01.10.2014   
               99210 0            Kredit    2299             EUR  15.05.2015   
               99216 0            Kredit    5000             EUR  15.11.2013   
               99208 2            Kredit    4869             EUR  01.06.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 4              NaN                    NaN           3   
                     5              NaN                    NaN           4   
               99210 0              NaN                    NaN           1   
               99216 0              NaN                    NaN           1   
               99208 2              NaN                    NaN           2   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 4                                  NaN  hauptmerkmal   
                     5                                  NaN  hauptmerkmal   
               99210 0                                  NaN  hauptmerkmal   
               99216 0                                  NaN  hauptmerkmal   
               99208 2                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 4               KR        M        69            False   
                     5               KR        M        84            False   
               99210 0               KR        M        36            False   
               99216 0               KR        M        60            False   
               99208 2               KR        M        36            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  
dwh_country_id id    record                                                   
1              99197 4                    False                        False  
                     5                    False                        False  
               99210 0                    False                        False  
               99216 0                    False                        False  
               99208 2                    False                        False

In [123]:
net_loans_exclude_US_outstanding.Beschreibung.unique()

array(['Kredit', 'Leasing', u'Saldo nach Gesamtf\xe4lligstellung',
       u'\xdcbergabe an ein Inkassounternehmen/eine interne Inkassostelle zur Beitreibung',
       'titulierter Saldo', 'Saldo', 'Forderungsverkauf',
       'uneinbringliche titulierte Forderung / Einzug unwirtschaftlich',
       'Widerspruch zum Saldo',
       u'R\xfcckgabe vom Inkassounternehmen/von einer internen Inkassostelle an den Gl\xe4ubiger'], dtype=object)

In [34]:
12+7-9+1

11

In [245]:
a=net_loans_exclude_US_outstanding[net_loans_exclude_US_outstanding.Ratenart=='J']
a.to_clipboard()

In [169]:
net_loans_exclude_US_outstanding.Ratenart.unique()

array(['M', nan, 'J'], dtype=object)

In [185]:
91//12

7

In [191]:
net_loans_exclude_US_outstanding.loc[KR_select & (net_loans_exclude_US_outstanding.Ratenart=='J')]

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99315 0            Kredit   60000             EUR  30.12.2007   
               96453 0            Kredit    7414             EUR  31.03.2001   
               96780 2            Kredit   20250             EUR  28.02.2010   
               98778 0            Kredit    9900             EUR  31.05.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99315 0              NaN                    NaN           1   
               96453 0              NaN                    NaN           1   
               96780 2              NaN                    NaN           3   
               98778 0              NaN                    NaN           1   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99315 0                                  NaN  hauptmerkmal   
               96453 0                                  NaN  hauptmerkmal   
               96780 2                                  NaN  hauptmerkmal   
               98778 0                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart  Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                    
1              99315 0               KR        J          8            False   
               96453 0               KR        J         15            False   
               96780 2               KR        J          8            False   
               98778 0               KR        J         10            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99315 0                    False                        False   
               96453 0                    False                        False   
               96780 2                    False                        False   
               98778 0                    False                        False   

                                  Date  paid_installments  installment  
dwh_country_id id    record                                             
1              99315 0      2007-12-30                  7         9750  
               96453 0      2001-03-31                 14          642  
               96780 2      2010-02-28                  5         3290  
               98778 0      2014-05-31                  1         1287

In [78]:
net_loans_exclude_US_outstanding.xs(99550,level='id').to_clipboard()

In [69]:
lr=118813
loan_requests[loan_requests.id_loan_request==lr]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data


In [206]:
id=99315
loan_requests[loan_requests.id==id]

,id_loan_request,loan_request_nr,fk_user,id
62,122080,767115004,137604,99315


In [67]:
net_loans_exclude_US_outstanding.head()

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99197 4            Kredit    5364             EUR  01.09.2014   
                     5            Kredit   27197             EUR  01.10.2014   
               99210 0            Kredit    2299             EUR  15.05.2015   
               99216 0            Kredit    5000             EUR  15.11.2013   
               99208 2            Kredit    4869             EUR  01.06.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 4              NaN                    NaN           3   
                     5              NaN                    NaN           4   
               99210 0              NaN                    NaN           1   
               99216 0              NaN                    NaN           1   
               99208 2              NaN                    NaN           2   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 4                                  NaN  hauptmerkmal   
                     5                                  NaN  hauptmerkmal   
               99210 0                                  NaN  hauptmerkmal   
               99216 0                                  NaN  hauptmerkmal   
               99208 2                                  NaN  hauptmerkmal   

                            Merkmalcode    ...     not_creditworthy_all  \
dwh_country_id id    record                ...                            
1              99197 4               KR    ...                    False   
                     5               KR    ...                    False   
               99210 0               KR    ...                    False   
               99216 0               KR    ...                    False   
               99208 2               KR    ...                    False   

                             paid_installments Ratenzahl_Month installment  \
dwh_country_id id    record                                                  
1              99197 4                      11              69         101   
                     5                      10              84         420   
               99210 0                       3              36          83   
               99216 0                      21              60         108   
               99208 2                      14              36         175   

                            loan_gross_amount loan_gross_outstanding_amount  \
dwh_country_id id    record                                                   
1              99197 4                   6969                          5858   
                     5                  35280                         31080   
               99210 0                   2988                          2739   
               99216 0                   6480                          4212   
               99208 2                   6300                          3850   

                            outstanding_installments  interest  \
dwh_country_id id    record                                      
1              99197 4                            58      1605   
                     5                            74      8083   
               99210 0                            33       689   
               99216 0                            39      1480   
               99208 2                            22      1431   

                             interest_allowance  net_value  
dwh_country_id id    record                                 
1              99197 4                   986.93    4871.07  
                     5                  5626.40   25453.60  
               99210 0                   450.0

In [50]:
%run indebtedness.py

In [239]:
merkmale.xs(96439,level='id')

Beschreibung  Betrag  \
dwh_country_id record                                            
1              0              Kredit mit Endfälligkeit     100   
               1                              Erledigt     NaN   
               2              Kredit mit Endfälligkeit     100   
               3                              Erledigt     NaN   
               4              Kredit mit Endfälligkeit     100   
               5                              Erledigt     NaN   
               6              Kredit mit Endfälligkeit     200   
               7                              Erledigt     NaN   
               8              Kredit mit Endfälligkeit     200   
               9                              Erledigt     NaN   
               10             Kredit mit Endfälligkeit     200   
               11                             Erledigt     NaN   
               12             Kredit mit Endfälligkeit     200   
               13      Saldo nach Gesamtfälligstellung     300   
               14                            Girokonto     NaN   

                      Betrag_waehrung        Datum Kontonummer  \
dwh_country_id record                                            
1              0                  EUR  UNBEFRISTET         NaN   
               1                  NaN   05.08.2014         NaN   
               2                  EUR  UNBEFRISTET         NaN   
               3                  NaN   05.09.2014         NaN   
               4                  EUR  UNBEFRISTET         NaN   
               5                  NaN   02.10.2014         NaN   
               6                  EUR  UNBEFRISTET         NaN   
               7                  NaN   03.11.2014         NaN   
               8                  EUR  UNBEFRISTET         NaN   
               9                  NaN   04.12.2014         NaN   
               10                 EUR  UNBEFRISTET         NaN   
               11                 NaN   06.01.2015         NaN   
               12                 EUR  UNBEFRISTET         NaN   
               13                 EUR   26.03.2015         NaN   
               14                 NaN          NaN         NaN   

                      Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id record                                      
1              0                         NaN           1   
               1                         NaN           1   
               2                         NaN           2   
               3                         NaN           2   
               4                         NaN           3   
               5                         NaN           3   
               6                         NaN           4   
               7                         NaN           4   
               8                         NaN           5   
               9                         NaN           5   
               10                        NaN           6   
               11                        NaN           6   
               12                        NaN           7   
               13                        NaN           7   
               14                        NaN           8   

                      Merkmal_merkmalOhneGeburtsdatum         Merkmal_typ  \
dwh_country_id record                                                       
1              0                                  NaN        hauptmerkmal   
               1                                  NaN  erledigungsmerkmal   
               2                                  NaN        hauptmerkmal   
               3                                  NaN  erledigungsmerkmal   
               4                                  NaN        hauptmerkmal   
               5                                  NaN  erledigungsmerkmal   
               6                                  NaN        hauptmerkmal   
               7                                  NaN  erledigungsmerkmal   
     

In [240]:
loan_requests[loan_requests.id==96439]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount
348,118766,484010163,134308,96439,3000


In [52]:
tod

'20150803'

In [47]:
net_loans_exclude_US_outstanding_user = calc_net_loans(merkmale, '2015-08-03')

In [48]:
loan_requests_indebtedness = calc_indebtedness(loan_requests1, not_creditworthy_user, 
                      additional_limit_outstanding_user,  
                      LOC_outstanding_user, 
                      net_loans_exclude_US_outstanding_user)

In [51]:
tod=today_str()

In [55]:
loan_requests_indebtedness.to_csv('loan_requests_indebtedness_all_user_{}.csv'.format(tod),encoding='UTF-8')

In [54]:
scoring.to_csv('scoring_{}.csv'.format(tod),encoding='UTF-8')
loan_requests1.to_csv('loan_requests1_{}.csv'.format(tod),encoding='UTF-8')
#scoring_ind.to_csv('scoring_ind_{}.csv'.format(tod),encoding='UTF-8')
merkmale.to_csv('merkmale_{}.csv'.format(tod),encoding='UTF-8')
#all_user.to_csv('all_user_{}.csv'.format(tod),encoding='UTF-8')
loan_requests_indebtedness.to_csv('loan_requests_indebtedness_all_user_{}.csv'.format(tod),encoding='UTF-8')

NameError: name 'lr_all_user' is not defined

In [ ]:
/html/body/table/tbody
<tr><th colspan="2">Indebtedness</th><th colspan="2">Indebtedness Ratio</th></tr>

<tbody><tr><th colspan="2">Indebtedness</th><th colspan="2">Indebtedness Ratio</th></tr>
        <tr><td>Net Debt</td><td>13.555,91 €</td><td>Debt Ratio</td><td>9,68%</td></tr>
        <tr><td>Net Loan Values</td><td>10.555,91 €</td><td>Debt Calculation</td><td>13555.9121 / 1400</td></tr>
        <tr><td>Net Lendico Loan</td><td>3.000,00 €</td><td></td><td></td></tr>
        <tr><td>Line of Credit</td><td>0,00 €</td><td></td><td></td></tr>
        <tr><td>Additional Limit</td><td>0,00 €</td><td></td><td></td></tr>
        <tr><td>Net Income</td><td>1.400,00 €</td><td></td><td></td></tr>
        <tr><td>Creditworthy?</td><td>Yes</td><td></td><td></td></tr>
    </tbody>
    
//th[text()='2']/following-sibling::td[1]
    

In [61]:
merkmale.head()

Beschreibung  Betrag Betrag_waehrung  \
dwh_country_id id    record                                             
1              99213 0       Abwicklungskonto     NaN             NaN   
                     1                  Saldo    3671             EUR   
                     2                  Saldo    3715             EUR   
                     3                  Saldo    3760             EUR   
                     4                  Saldo    3850             EUR   

                                  Datum Kontonummer Merkmal_eigenesMerkmal  \
dwh_country_id id    record                                                  
1              99213 0       02.12.2006         NaN                    NaN   
                     1       31.03.2011         NaN                    NaN   
                     2       30.06.2011         NaN                    NaN   
                     3       30.09.2011         NaN                    NaN   
                     4       31.03.2012         NaN                    NaN   

                             Merkmal_id Merkmal_merkmalOhneGeburtsdatum  \
dwh_country_id id    record                                               
1              99213 0                1                             NaN   
                     1                1                             NaN   
                     2                1                             NaN   
                     3                1                             NaN   
                     4                1                             NaN   

                              Merkmal_typ Merkmalcode Ratenart  Ratenzahl  \
dwh_country_id id    record                                                 
1              99213 0       hauptmerkmal          KW      NaN        NaN   
                     1       untermerkmal          SD      NaN        NaN   
                     2       untermerkmal          SD      NaN        NaN   
                     3       untermerkmal          SD      NaN        NaN   
                     4       untermerkmal          SD      NaN        NaN   

                                  Date not_creditworthy not_creditworthy_US  \
dwh_country_id id    record                                                   
1              99213 0      2006-12-02             True                 NaN   
                     1      2011-03-31            False                 NaN   
                     2      2011-06-30            False                 NaN   
                     3      2011-09-30            False                 NaN   
                     4      2012-03-31            False                 NaN   

                            not_creditworthy_KRMLKWKX_US not_creditworthy_all  
dwh_country_id id    record                                                    
1              99213 0                             False                 True  
                     1                             False                False  
                     2                             False                False  
                     3                             False                False  
                     4                             False                False

In [63]:
lr=126064
loan_requests1[loan_requests1.id_loan_request==lr]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data,indebt:actual,indebt:expected,indebt:operator,indebt:status
420,126064,331860036,141645,102485,5000,1400,null,NaN,NaN,None,None
425,126064,331860036,141645,102485,5000,1400,"[{""status"":true,""actual"":""salaried"",""expected""...",11.11,14,<=,True


In [76]:
merkmale.xs(99550,level='id').to_clipboard()

In [75]:
lr_all_user[lr_all_user['indebt:actual'].notnull()].to_clipboard()

In [316]:
loan_requests1.shape

(3340, 11)

In [41]:
(loan_requests.postcheck_data!='null').sum()

224

In [244]:
lr_all_user[lr_all_user.id_loan_request==120973].T.to_clipboard()

In [217]:
net_loans_exclude_US_outstanding_user.head()

Betrag  loan_gross_amount  \
dwh_country_id id                                 
1              96442   33975              44016   
               96443    8844                NaN   
               96444     183                230   
               96445   28749              21720   
               96446   20000              25956   

                      loan_gross_outstanding_amount  interest  \
dwh_country_id id                                               
1              96442                          35894     10041   
               96443                            NaN       NaN   
               96444                              0        47   
               96445                           7610      4943   
               96446                          24411      5956   

                      interest_allowance  net_value  
dwh_country_id id                                    
1              96442             6302.86   29591.14  
               96443                 NaN        NaN  
               96444                5.13       0.00  
               96445              492.99    7117.01  
               96446             4754.79   19656.21

In [178]:
net_loans_exclude_US_outstanding.loc[KR_select].head()

Beschreibung  Betrag Betrag_waehrung       Datum  \
dwh_country_id id    record                                                    
1              99197 4            Kredit    5364             EUR  01.09.2014   
                     5            Kredit   27197             EUR  01.10.2014   
               99210 0            Kredit    2299             EUR  15.05.2015   
               99216 0            Kredit    5000             EUR  15.11.2013   
               99208 2            Kredit    4869             EUR  01.06.2014   

                            Kontonummer Merkmal_eigenesMerkmal  Merkmal_id  \
dwh_country_id id    record                                                  
1              99197 4              NaN                    NaN           3   
                     5              NaN                    NaN           4   
               99210 0              NaN                    NaN           1   
               99216 0              NaN                    NaN           1   
               99208 2              NaN                    NaN           2   

                            Merkmal_merkmalOhneGeburtsdatum   Merkmal_typ  \
dwh_country_id id    record                                                 
1              99197 4                                  NaN  hauptmerkmal   
                     5                                  NaN  hauptmerkmal   
               99210 0                                  NaN  hauptmerkmal   
               99216 0                                  NaN  hauptmerkmal   
               99208 2                                  NaN  hauptmerkmal   

                            Merkmalcode Ratenart Ratenzahl not_creditworthy  \
dwh_country_id id    record                                                   
1              99197 4               KR        M        69            False   
                     5               KR        M        84            False   
               99210 0               KR        M        36            False   
               99216 0               KR        M        60            False   
               99208 2               KR        M        36            False   

                            not_creditworthy_US not_creditworthy_KRMLKWKX_US  \
dwh_country_id id    record                                                    
1              99197 4                    False                        False   
                     5                    False                        False   
               99210 0                    False                        False   
               99216 0                    False                        False   
               99208 2                    False                        False   

                                  Date  paid_installments  
dwh_country_id id    record                                
1              99197 4      2014-09-01                 11  
                     5      2014-10-01                 10  
               99210 0      2015-05-15                  3  
               99216 0      2013-11-15                 21  
               99208 2      2014-06-01                 14

In [306]:
loan_requests[loan_requests.id_loan_request==122466]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data
6687,122466,44200180,7001,99634,1500,2198,"[{""status"":true,""actual"":""retired"",""expected"":..."
6693,122466,44200180,7001,99634,1500,2198,null


In [125]:
z1=match_id(net_loans_exclude_US_outstanding,net_loans_exclude_US_outstanding.Beschreibung==u'R\xfcckgabe vom Inkassounternehmen/von einer internen Inkassostelle an den Gl\xe4ubiger')
z1

Beschreibung  \
dwh_country_id id     record                                                      
1              100654 3                                                  Kredit   
                      4                         Saldo nach Gesamtfälligstellung   
                      5                                                   Saldo   
                      6                                                   Saldo   
                      7                                                   Saldo   
                      8                                                   Saldo   
                      9                                                   Saldo   
                      10                                                  Saldo   
                      11                                                  Saldo   
                      12                                                  Saldo   
                      13                                                  Saldo   
                      14                                                  Saldo   
                      15                                                  Saldo   
                      16                                                  Saldo   
                      17                                                  Saldo   
                      18                                                  Saldo   
                      19                                                  Saldo   
                      20                                                  Saldo   
                      21                                                  Saldo   
                      22                                                  Saldo   
                      23                                                  Saldo   
                      24                                                  Saldo   
                      25                                                  Saldo   
                      26                                                  Saldo   
                      27                                                  Saldo   
                      28                                                  Saldo   
                      29                                                  Saldo   
                      30                                                  Saldo   
                      31                                                  Saldo   
                      32                                                  Saldo   
                      33                                                  Saldo   
                      34                                                  Saldo   
                      35                                                  Saldo   
                      36                                                  Saldo   
                      37                                                  Saldo   
                      38                                                  Saldo   
                      39                                                  Saldo   
                      40      Rückgabe vom Inkassounternehmen/von einer inte...   

                              Betrag Betrag_waehrung       Datum Kontonummer  \
dwh_country_id id     record                                                   
1              100654 3        35023             EUR  15.04.2010         NaN   
                      4        17388             EUR  24.05.2012         NaN   
                      5        17402             EUR  31.05.2012         NaN   
                      6        17432             EUR  13.06.2012         NaN   
                      7        17458             EUR  16.08.2012         NaN   
                      8        17518             EUR  12.09.2012         NaN   
                      9        17587             EUR  12.10.2012         NaN   
                      10  

In [ ]:
net_loans_exclude_US_outstanding_user.xs()

# Web Requests

In [ ]:
import logging
import httplib
httplib.HTTPConnection.debuglevel = 1

logging.basicConfig() # you need to initialize logging, otherwise you will not see anything from requests
logging.getLogger().setLevel(logging.DEBUG)
requests_log = logging.getLogger("requests.packages.urllib3")
requests_log.setLevel(logging.DEBUG)


In [56]:
payload={'_username':'sean.violante@lendico.de',
         '_password': '7642lottA!!',
        'login':'Login'}

headers = {'User-Agent': 'Mozilla/5.0'}
with requests.Session() as s:
    s.auth=('lendico_beta', 'p2pLend4every1')
    r1 = s.get('https://admin.lendico.de/admin')
    p = s.post('https://admin.lendico.de/admin/login_check', headers=headers, data=payload)
    # print the html returned or something more intelligent to see if it's a successful login page.
    "
    # An authorised request.
    r = s.get('https://admin.lendico.de/admin/loan-request/display-indebtedness-details/122367')
    print r.text

<!DOCTYPE html>
<html>
<head><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o?o:n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({QJf3ax:[function(e,n){function t(e){function n(n,t,a){e&&e(n,t,a),a||(a={});for(var u=c(n),f=u.length,s=i(a,o,r),p=0;f>p;p++)u[p].apply(s,t);return s}function a(e,n){f[e]=c(e).concat(n)}function c(e){return f[e]||[]}function u(){return t(n)}var f={};return{on:a,emit:n,create:u,listeners:c,_events:f}}function r(){return{}}var o="nr@context",i=e("gos");n.exports=t()},{gos:"7eSDFh"}],ee:[function(e,n){n.exports=e("QJf3ax")},{}],3:[function(e,n){function t(e){return function(){r(e,[(new Date).getTime()].concat(i(arguments)))}}var r=e("handle"),o=e(1),i=e(2);"undefined"==typeof window.newrelic&&(newrelic=window.NREUM);var a=["setPag

In [77]:
(loan_requests_indebtedness.postcheck_data!='null').sum()

217

In [80]:
lrs=loan_requests_indebtedness.id_loan_request[loan_requests_indebtedness.postcheck_data!='null'].unique()

In [114]:
df1.columns

Index([u'Additional Limit',    u'Creditworthy?',   u'Line of Credit',
               u'Net Debt',       u'Net Income', u'Net Lendico Loan',
        u'Net Loan Values',               u'ad'],
      dtype='object')

In [82]:
lrs.shape

(215L,)

In [120]:
%run indebtedness.py

In [123]:
loan_requests_indebtedness.head()

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data,indebt:actual,indebt:expected,indebt:operator,...,add_Betrag,LOC_Betrag,net_loan_Betrag,loan_gross_amount,loan_gross_outstanding_amount,interest,interest_allowance,net_value,net_debt,debt_ratio
0,16349,504978105,26190,98005,5000,1784,null,NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,38589.00,21.630605
1,16349,504978105,26190,98005,5000,1784,"[{""status"":true,""actual"":""salaried"",""expected""...",NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,38589.00,21.630605
2,120607,466264165,26190,98005,20000,1784,null,NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,53589.00,30.038677
3,120607,466264165,26190,98005,20000,1784,"[{""status"":true,""actual"":""salaried"",""expected""...",NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,53589.00,30.038677
4,35214,951059135,46471,101247,3000,2100,null,NaN,NaN,None,...,NaN,NaN,42931,55620,28847,12689,3415.04,25439.38,28439.38,13.542562


In [124]:
loan_requests_indebtedness_web=loan_requests_indebtedness.merge(df2.reset_index(),on='id_loan_request')

In [125]:
loan_requests_indebtedness_web.to_clipboard()

In [122]:
df2.head()

,Additional Limit,Creditworthy?,Debt Ratio,Line of Credit,Net Debt,Net Income,Net Lendico Loan,Net Loan Values
id_loan_request,,,,,,,,
16349,3700,1,21.65,0,38624.05,1784,5000,29924.05
120607,3700,1,30.06,0,53624.05,1784,20000,29924.05
35214,0,1,13.57,0,28487.86,2100,3000,25487.86
119079,0,1,15.95,0,33487.86,2100,8000,25487.86
126783,0,1,14.76,0,30987.86,2100,5500,25487.86


In [117]:
df1['CreditworthyX']=(df1['Creditworthy?']=='Yes')

In [119]:
%debug

> c:\users\sean violante\documents\projects\lendico\projects\indebtedness\src\indebtedness.py(350)get_web_indebtedness()
    349     del df['Debt Calculation']
--> 350     data['Creditworthy?']=(data['Creditworthy?']=='Yes')
    351     for c in cols:

ipdb> q


In [121]:
%time df2=get_web_indebtedness(lrs)

Wall time: 1min 14s


In [99]:
df1=pd.DataFrame.from_records(raw,index='id_loan_request')

In [102]:
df1['ad']=df1['Additional Limit'].astype(np.double)


In [108]:
del df1[ 'Debt Ratio']

In [109]:
df1.astype(np.double).head()

ValueError: could not convert string to float: Yes

In [81]:
print loan_requests1.shape
print loan_requests_indebtedness.shape

(579, 11)
(446, 25)


In [78]:
loan_requests_indebtedness.head()

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data,indebt:actual,indebt:expected,indebt:operator,...,add_Betrag,LOC_Betrag,net_loan_Betrag,loan_gross_amount,loan_gross_outstanding_amount,interest,interest_allowance,net_value,net_debt,debt_ratio
0,16349,504978105,26190,98005,5000,1784,null,NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,38589.00,21.630605
1,16349,504978105,26190,98005,5000,1784,"[{""status"":true,""actual"":""salaried"",""expected""...",NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,38589.00,21.630605
2,120607,466264165,26190,98005,20000,1784,null,NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,53589.00,30.038677
3,120607,466264165,26190,98005,20000,1784,"[{""status"":true,""actual"":""salaried"",""expected""...",NaN,NaN,None,...,3700,NaN,47484,61608,33436,14124,3547.00,29889.00,53589.00,30.038677
4,35214,951059135,46471,101247,3000,2100,null,NaN,NaN,None,...,NaN,NaN,42931,55620,28847,12689,3415.04,25439.38,28439.38,13.542562


In [60]:
soup = BeautifulSoup(r.text, "html")
table = soup.find('table', {'class': 'table table-striped'})


In [61]:
table

<table class="table table-striped">
<tr><th colspan="2">Indebtedness</th><th colspan="2">Indebtedness Ratio</th></tr>
<tr><td>Net Debt</td><td>1.500,00 €</td><td>Debt Ratio</td><td>0,94%</td></tr>
<tr><td>Net Loan Values</td><td>0,00 €</td><td>Debt Calculation</td><td>1500 / 1600</td></tr>
<tr><td>Net Lendico Loan</td><td>1.500,00 €</td><td></td><td></td></tr>
<tr><td>Line of Credit</td><td>0,00 €</td><td></td><td></td></tr>
<tr><td>Additional Limit</td><td>0,00 €</td><td></td><td></td></tr>
<tr><td>Net Income</td><td>1.600,00 €</td><td></td><td></td></tr>
<tr><td>Creditworthy?</td><td>Yes</td><td></td><td></td></tr>
</table>

In [ ]:
th = table.find('th', text='Indebtedness:')


In [62]:
tdList = table.findAll('td')

In [74]:

td_key=map(lambda x: x.text,tdList[0::2])
td_value=map(lambda x: x.text,tdList[1::2])
td_value=[re.sub(u'[.%\u20ac]','',td) for td in td_value]
td_value=[re.sub(u' \u20ac','',td) for td in td_value]
td_value=[re.sub(',','.',td) for td in td_value]
data=dict(zip(td_key,td_value))
del data['']
data

{u'Additional Limit': u'0.00 ',
 u'Creditworthy?': u'Yes',
 u'Debt Calculation': u'1500 / 1600',
 u'Debt Ratio': u'0.94',
 u'Line of Credit': u'0.00 ',
 u'Net Debt': u'1500.00 ',
 u'Net Income': u'1600.00 ',
 u'Net Lendico Loan': u'1500.00 ',
 u'Net Loan Values': u'0.00 '}

In [67]:
data

{u'': u'',
 u'Additional Limit': u'0,00 \u20ac',
 u'Creditworthy?': u'Yes',
 u'Debt Calculation': u'1500 / 1600',
 u'Debt Ratio': u'0,94%',
 u'Line of Credit': u'0,00 \u20ac',
 u'Net Debt': u'1.500,00 \u20ac',
 u'Net Income': u'1.600,00 \u20ac',
 u'Net Lendico Loan': u'1.500,00 \u20ac',
 u'Net Loan Values': u'0,00 \u20ac'}

In [69]:
data

{u'Additional Limit': u'0,00 \u20ac',
 u'Creditworthy?': u'Yes',
 u'Debt Calculation': u'1500 / 1600',
 u'Debt Ratio': u'0,94%',
 u'Line of Credit': u'0,00 \u20ac',
 u'Net Debt': u'1.500,00 \u20ac',
 u'Net Income': u'1.600,00 \u20ac',
 u'Net Lendico Loan': u'1.500,00 \u20ac',
 u'Net Loan Values': u'0,00 \u20ac'}

## Investigation

In [ ]:
merkmale[merkmale.Merkmal_merkmalOhneGeburtsdatum=='ja'].to_clipboard(encoding='UTF-8')

In [ ]:
identify which 

In [348]:
d.head()

,dwh_country_id,id,record,Beschreibung,Betrag,Betrag_waehrung,Datum,Kontonummer,Merkmal_eigenesMerkmal,Merkmal_id,Merkmal_merkmalOhneGeburtsdatum,Merkmal_typ,Merkmalcode,Ratenart,Ratenzahl,not_creditworthy
0,1,99207,10,XX,NaN,NaN,NaN,NaN,NaN,11,NaN,hauptmerkmal,XX,NaN,NaN,False
1,1,99207,11,Rahmenkredit,500,EUR,UNBEFRISTET,NaN,NaN,11,NaN,untermerkmal,RK,NaN,NaN,False
2,1,99207,12,XX,NaN,NaN,NaN,NaN,NaN,12,NaN,hauptmerkmal,XX,NaN,NaN,False
3,1,99207,13,Rahmenkredit,300,EUR,UNBEFRISTET,NaN,NaN,12,NaN,untermerkmal,RK,NaN,NaN,False
4,1,99219,12,XX,NaN,NaN,NaN,NaN,NaN,10,NaN,hauptmerkmal,XX,NaN,NaN,False


In [86]:
loan_requests1[loan_requests1.fk_user==137886]

,id_loan_request,loan_request_nr,fk_user,id,principal_amount,net_income,postcheck_data,indebt:actual,indebt:expected,indebt:operator,indebt:status
349,122362,342270959,137886,99550,1500,865,null,NaN,NaN,None,None
350,122362,342270959,137886,99550,1500,865,"[{""status"":false,""actual"":""student"",""expected""...",1.20,14,<=,True
367,123271,626638404,137886,99550,1000,865,"[{""status"":false,""actual"":""student"",""expected""...",1.73,14,<=,True
370,123271,626638404,137886,99550,1000,865,null,NaN,NaN,None,None


In [397]:
map(lambda x:pd.Series(x[1].unique(),name=x[0]),merkmale[
        [u'Beschreibung',  u'Betrag_waehrung','Datum', 
         'Merkmal_eigenesMerkmal','Merkmal_merkmalOhneGeburtsdatum',
         'Merkmal_typ',
         u'Merkmalcode',
       u'Ratenart', u'Ratenzahl' ]].iteritems())

[0                                      Abwicklungskonto
 1                                                 Saldo
 2                                     titulierter Saldo
 3     Gläubigerbefriedigung ausgeschlossen/nicht nac...
 4                                                Kredit
 5                                              Erledigt
 6                                             Girokonto
 7                                           Kreditkarte
 8                          Kreditkarte mit Kreditrahmen
 9                                    Anfrage zum Kredit
 10    Schuldner ist seiner Pflicht zur Abgabe der Ve...
 11               grundpfandrechtlich gesicherter Kredit
 12                               Anfrage zur Bürgschaft
 13                                              Leasing
 14    Investitionskredit (für Selbständige und Klein...
 15                                                   XX
 16                                         Rahmenkredit
 17                          Mi

In [87]:
scoring[scoring.fk_user==137886]

,dwh_country_id,id,fk_user,request,response,provider_name,is_failed,ignore_for_scoring,score,data_accessible_by,created_at,updated_at,dwh_created,dwh_last_modified,dwh_row_hash,is_latest
2381,1,99550,137886,"<?xml version=""1.0"" encoding=""UTF-8""?>\n<Schuf...","s:6449:""<?xml version=""1.0"" encoding=""UTF-8"" s...",score.provider.schufa,0,0,7596,4.611686e+18,2015-07-11 21:10:02,2015-07-14 14:08:51,2015-07-11 21:14:06.470314,2015-07-14 14:10:47.749388,2ddc164d16d9ceb3c9617bc3b17fab20,1


In [293]:
merkmale.Merkmal_typ.unique()

array(['hauptmerkmal', 'untermerkmal', 'einzelmerkmal',
       'erledigungsmerkmal'], dtype=object)

In [219]:
scoring_ind['response_x'][(scoring_ind['response_root'].isnull())].unique()

array([u'ANONYMOUS_REQUEST', u'deleted_user'], dtype=object)

In [258]:
merkmale.to_clipboard()

In [214]:
scoring_ind.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4305 entries, (1, 99213) to (1, 100774)
Data columns (total 18 columns):
fk_user               4305 non-null int64
request               4305 non-null object
response              4305 non-null object
provider_name         4305 non-null object
is_failed             4305 non-null int64
ignore_for_scoring    4305 non-null int64
score                 4305 non-null object
data_accessible_by    4305 non-null float64
created_at            4305 non-null datetime64[ns]
updated_at            4305 non-null datetime64[ns]
dwh_created           4305 non-null datetime64[ns]
dwh_last_modified     4305 non-null datetime64[ns]
dwh_row_hash          4305 non-null object
is_latest             4305 non-null int64
response_chk          4305 non-null object
response_x            4305 non-null object
response_len          4305 non-null int64
response_root         3707 non-null object
dtypes: datetime64[ns](4), float64(1), int64(5), object(8)
memory usage: 65

In [205]:
merkmale.iloc[3]

,Beschreibung,Betrag,Betrag_waehrung,Datum,Merkmal_typ,Merkmalcode,Merkmal_id
0,Girokonto,NaN,NaN,NaN,hauptmerkmal,GI,1
1,Abwicklungskonto,NaN,NaN,06.05.2013,hauptmerkmal,KW,2
2,Saldo,135,EUR,06.05.2013,untermerkmal,SD,2
3,Saldo,153,EUR,28.10.2013,untermerkmal,SD,2
